In [1]:
# Libraries
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

# Load ports
capitanias_path = '../../data/puertos/raw/'
capitanias_filename = 'Capitanias Latitude Longitude_edited.csv'

path_file = os.path.join(capitanias_path, capitanias_filename)

cap_latlon = pd.read_csv(path_file) 
cap_pd = cap_latlon[['id_capitania', 'capitania', 'caleta', 'lat_decimal', 'lon_decimal']]
cap_pd = cap_pd.rename(columns={'lat_decimal':'latitude', 'lon_decimal':'longitude'})
cap_pd.loc[:, 'latitude'] = cap_pd['latitude']*-1
cap_pd.loc[:, 'longitude'] = cap_pd['longitude']*-1
cap_pd = cap_pd.drop_duplicates()

cap_pd = cap_pd.rename(columns={'caleta':'port_name'})
cap_pd['port_name'] = cap_pd['port_name'].str.replace('  ',' ')
cap_pd['port_name'] = cap_pd['port_name'].str.strip()
cap_pd = cap_pd[cap_pd['port_name'] != 'caleta']
cap_pd['port_name'] = cap_pd['port_name'].str.replace('Desembarcadero pesquero artesanal','DPA')
cap_pd['port_name'] = cap_pd['port_name'].str.replace('desembarcadero pesquero artesanal','DPA')
cap_pd['port_name'] = cap_pd['port_name'].str.upper()
cap_pd['port_name'] = cap_pd['port_name'].str.replace(' ','_')
cap_pd['capitania'] = cap_pd['capitania'].str.upper()
cap_pd['capitania'] = cap_pd['capitania'].str.replace('CAPITANIA DE ','')
cap_pd['capitania'] = cap_pd['capitania'].str.replace(' ','_')

df_ports = cap_pd.copy()

xr_ports = xr.Dataset(
    {
        'latitude': ('port', df_ports['latitude'].values),
        'longitude': ('port', df_ports['longitude'].values),
        'capitania': ('port', df_ports['capitania'].values),
        'id_capitania': ('port', df_ports['id_capitania'].values)
    },
    coords={
        'port': df_ports['port_name'].values
    }
)

ports_path = '../../data/puertos/processed/'
ports_filename = "xr_ports.nc"
file_path = os.path.join(ports_path, ports_filename)

if not os.path.exists(file_path):
    xr_ports.to_netcdf(file_path)
    print("File saved successfully.")
else:
    print("The file already exists. It was not overwritten.")

    

The file already exists. It was not overwritten.
